<a href="https://colab.research.google.com/github/vivirocha/Bootcamp_DataScience/blob/main/DesenvolvendoSolu%C3%A7%C3%B5esUtilizandoApacheSpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Bootcamp Cientista de Dados - IGTI**

Trabalho 1 - Módulo 2

*Desenvolvendo Soluções Utilizando Apache Spark*


**Enunciado** <br> 
Dados do mercado financeiro são interessantes e ricos: cada ação negociada na bolsa de valores tem um preço que varia a cada dia. Você foi contratado como cientista de dados de uma empresa de Wall Street para criar modelos preditivos que, a partir da variação diária do preço das ações, consigam subsidiar e melhorar decisões de compra e venda de ações. Você disse que, como todo bom cientista de dados, gostaria de explorar os dados para entender suas características antes de criar qualquer modelo preditivo. <br>

Os dados estão disponíveis em https://www.kaggle.com/camnugent/sandp500/ por meio do arquivo all_stocks_5yr.csv. O arquivo contém, para cada dia e ação do S&P 500 (lista de 500 maiores empresas americanas), os seguintes dados: <br> 
● Date - no formato yy-mm-dd <br>
● Open - Preço da ação na abertura do mercado no dia, em dólares. <br>
● High - Maior preço alcançado naquele dia. <br>
● Low - Menor preço alcançado naquele dia. <br>
● Close - Preço da ação no fechamento do mercado no dia. <br>
● Volume - Número de ações vendidas / compradas.<br>
● Name - O nome da ação. <br>

Apesar do volume de dados ser pequeno, você decidiu usar o Apache Spark para processar os dados para aprender a ferramenta, e tendo em vista que a sua empresa disse que, em breve, obterá dados por minuto, e não por dia, e de todas as ações do planeta, não apenas dos Estados Unidos. Neste caso, uma ferramenta desenhada para lidar com big data será necessária, e você já quer estar com o código pronto.


In [48]:
#Instalando spark
!pip install spark

In [49]:
#Instalando pyspark
!pip install pyspark

In [50]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName('SolucoesSpark') \
                    .getOrCreate()

In [51]:
#Lendo o arquivo .csv e criando um dataframe dele.
df = spark.read.csv('all_stocks_5yr.csv', sep=',', header=True)

Para visualizarmos os dados do nosso dataframe, utilizaremos o comando **.show()**.

In [52]:
df.show(truncate=False)

+----------+-----+-----+-----+-----+--------+----+
|date      |open |high |low  |close|volume  |Name|
+----------+-----+-----+-----+-----+--------+----+
|2013-02-08|15.07|15.12|14.63|14.75|8407500 |AAL |
|2013-02-11|14.89|15.01|14.26|14.46|8882000 |AAL |
|2013-02-12|14.45|14.51|14.1 |14.27|8126000 |AAL |
|2013-02-13|14.3 |14.94|14.25|14.66|10259500|AAL |
|2013-02-14|14.94|14.96|13.16|13.99|31879900|AAL |
|2013-02-15|13.93|14.61|13.93|14.5 |15628000|AAL |
|2013-02-19|14.33|14.56|14.08|14.26|11354400|AAL |
|2013-02-20|14.17|14.26|13.15|13.33|14725200|AAL |
|2013-02-21|13.62|13.95|12.9 |13.37|11922100|AAL |
|2013-02-22|13.57|13.6 |13.21|13.57|6071400 |AAL |
|2013-02-25|13.6 |13.76|13.0 |13.02|7186400 |AAL |
|2013-02-26|13.14|13.42|12.7 |13.26|9419000 |AAL |
|2013-02-27|13.28|13.62|13.18|13.41|7390500 |AAL |
|2013-02-28|13.49|13.63|13.39|13.43|6143600 |AAL |
|2013-03-01|13.37|13.95|13.32|13.61|7376800 |AAL |
|2013-03-04|13.5 |14.07|13.47|13.9 |8174800 |AAL |
|2013-03-05|14.01|14.05|13.71|1

In [53]:
df.printSchema()


root
 |-- date: string (nullable = true)
 |-- open: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- close: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- Name: string (nullable = true)



Iremos renomear as colunas. <br>
Para renomearmos as colunas, usaremos o comando**.select(col('old_name').alias('new_name'))**. 

In [54]:
#Importando col a partir do pyspark.sql.functions
from pyspark.sql.functions import col

In [55]:
df = df.select(col("date").alias("data"), col("open").alias("abertura"), col("high").alias('maxima'), col('low').alias('minima'), col('close').alias('fechamento'), col('volume').alias('volume'), col('Name').alias('nome'))
df.show()

+----------+--------+------+------+----------+--------+----+
|      data|abertura|maxima|minima|fechamento|  volume|nome|
+----------+--------+------+------+----------+--------+----+
|2013-02-08|   15.07| 15.12| 14.63|     14.75| 8407500| AAL|
|2013-02-11|   14.89| 15.01| 14.26|     14.46| 8882000| AAL|
|2013-02-12|   14.45| 14.51|  14.1|     14.27| 8126000| AAL|
|2013-02-13|    14.3| 14.94| 14.25|     14.66|10259500| AAL|
|2013-02-14|   14.94| 14.96| 13.16|     13.99|31879900| AAL|
|2013-02-15|   13.93| 14.61| 13.93|      14.5|15628000| AAL|
|2013-02-19|   14.33| 14.56| 14.08|     14.26|11354400| AAL|
|2013-02-20|   14.17| 14.26| 13.15|     13.33|14725200| AAL|
|2013-02-21|   13.62| 13.95|  12.9|     13.37|11922100| AAL|
|2013-02-22|   13.57|  13.6| 13.21|     13.57| 6071400| AAL|
|2013-02-25|    13.6| 13.76|  13.0|     13.02| 7186400| AAL|
|2013-02-26|   13.14| 13.42|  12.7|     13.26| 9419000| AAL|
|2013-02-27|   13.28| 13.62| 13.18|     13.41| 7390500| AAL|
|2013-02-28|   13.49| 13

In [56]:
#Alterando o tipo das colunas
from pyspark.sql.types import StringType, DateType, FloatType
from pyspark.sql.types import BooleanType, IntegerType


In [57]:
df = df \
  .withColumn("data" ,
              df["data"]
              .cast(DateType()))   \
  .withColumn("abertura",
              df["abertura"]
              .cast(FloatType()))    \
   .withColumn("maxima",df["maxima"]
              .cast(FloatType()))    \
              .withColumn("minima", df["minima"].cast(FloatType()))    \
              .withColumn("fechamento",
              df["fechamento"]
              .cast(FloatType()))    \
              .withColumn("volume",
              df["volume"]
              .cast(IntegerType()))    \
              .withColumn("nome",
              df["nome"]
              .cast(StringType()))    \


In [58]:
#Visualizando os tipos de dados de cada coluna após a alteração.
df.printSchema()

root
 |-- data: date (nullable = true)
 |-- abertura: float (nullable = true)
 |-- maxima: float (nullable = true)
 |-- minima: float (nullable = true)
 |-- fechamento: float (nullable = true)
 |-- volume: integer (nullable = true)
 |-- nome: string (nullable = true)



Limitaremos a visualização de apenas 5 observações.

In [59]:
df.limit(5).toPandas()

,data,abertura,maxima,minima,fechamento,volume,nome
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


Iremos alterar a posição das colunas com o comando **.select()**.

In [60]:
df = df.select('nome', 'data', 'abertura', 'maxima', 'minima', 'fechamento', 'volume')
df.show()

+----+----------+--------+------+------+----------+--------+
|nome|      data|abertura|maxima|minima|fechamento|  volume|
+----+----------+--------+------+------+----------+--------+
| AAL|2013-02-08|   15.07| 15.12| 14.63|     14.75| 8407500|
| AAL|2013-02-11|   14.89| 15.01| 14.26|     14.46| 8882000|
| AAL|2013-02-12|   14.45| 14.51|  14.1|     14.27| 8126000|
| AAL|2013-02-13|    14.3| 14.94| 14.25|     14.66|10259500|
| AAL|2013-02-14|   14.94| 14.96| 13.16|     13.99|31879900|
| AAL|2013-02-15|   13.93| 14.61| 13.93|      14.5|15628000|
| AAL|2013-02-19|   14.33| 14.56| 14.08|     14.26|11354400|
| AAL|2013-02-20|   14.17| 14.26| 13.15|     13.33|14725200|
| AAL|2013-02-21|   13.62| 13.95|  12.9|     13.37|11922100|
| AAL|2013-02-22|   13.57|  13.6| 13.21|     13.57| 6071400|
| AAL|2013-02-25|    13.6| 13.76|  13.0|     13.02| 7186400|
| AAL|2013-02-26|   13.14| 13.42|  12.7|     13.26| 9419000|
| AAL|2013-02-27|   13.28| 13.62| 13.18|     13.41| 7390500|
| AAL|2013-02-28|   13.4

Quantos registros há na planilha?

Para respondermos essa questão podemos utilizar os comandos **.describe()** ou **.count()**.

In [61]:
df.describe().show()


+-------+------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|  nome|         abertura|           maxima|           minima|       fechamento|           volume|
+-------+------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|619040|           619029|           619032|           619032|           619040|           619040|
|   mean|  null| 83.0233343160683|83.77831069095873| 82.2560964206955|83.04376277067759|4321823.395568945|
| stddev|  null|97.37876902994715| 98.2075188299977|96.50742113042669|97.38974800596789|8693609.511967659|
|    min|     A|             1.62|             1.69|              1.5|             1.59|                0|
|    max|   ZTS|           2044.0|          2067.99|          2035.11|           2049.0|        618237630|
+-------+------+-----------------+-----------------+-----------------+-----------------+-----------------+



In [62]:
df.count()

619040

Quantos registros há na planilha para a ação da Apple (AAPL)?
<br> 


Para selecionarmos dados específicos de uma coluna específica utilizaremos as funções em conjunto: **select, where, count**. <br>
Então selecionaremos a coluna **nome**, onde exista nomes **igual a AAPL** *(ação da APPLE)* e **contaremos** quantas são.

In [63]:
df.select('nome').where(df.nome=="AAPL").count()

1259

Quantas empresas distintas têm registros nessa planilha?

Vamos importar a biblioteca countDistinct do pyspark.sql.functions.


In [64]:
#importando biblioteca para contagem de valores distintos 
from pyspark.sql.functions import countDistinct

Criaremos uma variável para inserir os valores distintos da coluna **nome** e visualizaremos com o comando **.show()**




In [65]:
empresasdistintas = df.select(countDistinct("nome"))
empresasdistintas.show()

+--------------------+
|count(DISTINCT nome)|
+--------------------+
|                 505|
+--------------------+



Com qual frequência o preço de uma ação no fechamento é maior do que o preço na abertura?

Criaremos uma coluna (**fech_abert**) para verificarmos em quais observações o preço de fechamento é maior do que o preço da abertura.


In [66]:
df = df.withColumn("fech_abert", col('fechamento') >col('abertura'))
df.limit(5).toPandas()

,nome,data,abertura,maxima,minima,fechamento,volume,fech_abert
0,AAL,2013-02-08,15.07,15.12,14.63,14.75,8407500,False
1,AAL,2013-02-11,14.89,15.01,14.26,14.46,8882000,False
2,AAL,2013-02-12,14.45,14.51,14.10,14.27,8126000,False
3,AAL,2013-02-13,14.30,14.94,14.25,14.66,10259500,True
4,AAL,2013-02-14,14.94,14.96,13.16,13.99,31879900,False


Vamos usar a função count para contarmos quantas vezes o fechamento foi maior que a abertura.

In [67]:
df.select('fech_abert').where(df.fech_abert=="True").count()


318970

Então, dividiremos o resultado pelo total dos eventos e arredondaremos o resultado.

In [68]:
porcentagem =  (318940/619040)*100
round(porcentagem, 2)

51.52

Qual o maior valor das ações da Apple (AAPL) na história? <br>
Vamos criar uma variável com o nome **apple** e inseriremos dentro dela apenas a valores encontrados em **AAPL**. Faremos um agrupamento para localizarmos apenas a cotação máxima da ação desta empresa.

In [69]:
#Filtrando pelo valor que desejamos encontrar.
apple = df[df['nome']=='AAPL']

In [70]:
#Agrupando por nome e procurando a maxima do valor que desejamos encontrar.
apple.groupby('nome').agg({'maxima': 'max'}).show()

+----+-----------+
|nome|max(maxima)|
+----+-----------+
|AAPL|      180.1|
+----+-----------+



Qual ação tem a maior volatilidade? Uma forma é medir o desvio-padrão do preço de fechamento de cada ação e considerar a ação de maior desvio-padrão.

In [71]:
#importando bibliotecas necessárias. 
from pyspark.sql.functions import stddev, sum, col, desc, max, min, mean


Utilizaremos um conjunto de funções combinadas para encontrarmos a ação com maior desvio padrão.<br>
**groupBy** para agrupar pelo nome da empresa. <br>
**agg(stddev().alias()** para salvar o resultado do desvio padrão da coluna fechamento na própria coluna fechamento. <br>
**sort(desc)** para ordenar por ordem decrescente. <br>
**show()** para visualizarmos os resultados.


In [72]:
df.groupBy("nome") \
  .agg(stddev("fechamento").alias("fechamento")) \
  .sort(desc("fechamento")) \
  .show()

+-----+------------------+
| nome|        fechamento|
+-----+------------------+
| PCLN|  320.533474036385|
| AMZN|282.50039426072794|
|GOOGL| 187.5738922672348|
| GOOG|166.42052806663057|
|  MTD| 131.3972196849655|
|  AZO| 130.7495964639748|
|  CMG| 130.2806239160575|
| EQIX|  95.2128051888174|
| REGN| 94.06934499598837|
| CHTR| 83.63699922834553|
| ISRG| 72.97599864131524|
| AVGO|  71.4162225390974|
| ULTA| 67.15184604914454|
|  NOC| 66.39592718115345|
|  SHW| 64.52058469695618|
|  LMT|  62.4120296184471|
|  BLK|62.336883810393154|
| NVDA| 59.62464185900208|
| ORLY|  59.0950803858885|
|  AGN| 59.02635016996632|
+-----+------------------+
only showing top 20 rows



Qual o dia com maior volume de negociação da bolsa?

In [73]:
df.groupBy("data") \
  .agg(sum("volume").alias("volume")) \
  .sort(desc("volume")) \
  .show()

+----------+----------+
|      data|    volume|
+----------+----------+
|2015-08-24|4607945196|
|2016-06-24|4367393052|
|2015-12-18|4124454411|
|2016-01-20|4087629753|
|2018-02-06|4072080890|
|2016-11-10|4060601612|
|2014-10-15|3993171524|
|2013-06-21|3983923288|
|2015-09-18|3962050449|
|2016-11-09|3915089371|
|2016-01-15|3787884056|
|2017-12-15|3786992731|
|2016-02-08|3759709109|
|2016-03-18|3755746256|
|2016-01-29|3740170664|
|2014-03-21|3611991561|
|2018-02-05|3598437288|
|2013-03-15|3569550899|
|2016-12-16|3564871264|
|2016-02-11|3547898496|
+----------+----------+
only showing top 20 rows



Qual a ação mais negociada da bolsa, em volume de transações? 


In [74]:
df.groupBy("nome") \
  .agg(sum("volume").alias("volume")) \
  .sort(desc("volume")) \
  .show()

+-----+------------+
| nome|      volume|
+-----+------------+
|  BAC|117884953591|
| AAPL| 68046305767|
|   GE| 53023644686|
|    F| 43388129992|
|   FB| 43258314899|
| MSFT| 42641653597|
|  AMD| 40941468586|
|   MU| 38082749687|
| INTC| 36922332801|
| CSCO| 36075826186|
|  PFE| 35527608853|
|    T| 31850598073|
|  CHK| 31421758366|
|CMCSA| 30630585625|
|  FCX| 29648463667|
|    C| 26044628798|
|  WFC| 23305954532|
|  JPM| 20885592857|
|   RF| 20645650500|
| NFLX| 20405882026|
+-----+------------+
only showing top 20 rows



Quantas ações começam com a letra “A”? <br>
Para descobrirmos quantos são os nomes das ações que começam com a letra A iremos fazer dois processos: <br>
**Primeiro:** criarmeos uma nova variável e usaremos o comando **.dropDuplicates** para excluirmos nos nomes duplicados da coluna **nome**. <br>
**Segundo:** usaremos o comando **filter** para filtrar os nomes que começam com a leta A (**startswith**).


In [75]:
startsA = df.dropDuplicates(['nome'])
print("Distinct count: "+str(startsA.count()))
startsA.show(truncate=False)

Distinct count: 505
+----+----------+--------+--------+-------+----------+--------+----------+
|nome|data      |abertura|maxima  |minima |fechamento|volume  |fech_abert|
+----+----------+--------+--------+-------+----------+--------+----------+
|ALXN|2013-02-08|93.11   |93.99   |92.9   |93.66     |1059844 |true      |
|GIS |2013-02-08|42.45   |42.66   |42.4101|42.6      |2092397 |true      |
|K   |2013-02-08|58.93   |59.35   |58.76  |58.91     |1669244 |false     |
|LEN |2013-02-08|39.4608 |39.5588 |38.6863|39.1765   |3701511 |false     |
|AIV |2013-02-08|27.4    |28.665  |27.2675|28.57     |1801890 |true      |
|AVY |2013-02-08|39.11   |39.43   |39.065 |39.25     |1003986 |true      |
|BF.B|2013-02-08|32.595  |32.695  |32.405 |32.54     |542314  |false     |
|AXP |2013-02-08|62.06   |62.33   |61.48  |61.8      |6734994 |false     |
|CI  |2013-02-08|61.7    |62.2199 |61.01  |61.93     |1638472 |true      |
|IRM |2013-02-08|34.63   |34.7    |33.88  |34.25     |1294887 |false     |
|INFO

In [76]:
startsA.filter(startsA.nome.startswith("A")).count()



59

Com qual frequência o preço mais alto do dia da ação também é o preço de fechamento? <br>
Para resolvermmos essa questão, criaremos uma nova coluna **alto_fech**, dentro dela conterá os dados verdadeiros caso o o preço mais alto do dia da ação seja o mesmo preço de fechamento e será falso caso o valor mais alto do dia seja diferente do valor de fechamento. <br>
Feito isso, selecionaremos os valores da coluna **alto_fech** onde exista valores **True**, contaremos quantos são e dividiremos pelo total de valores. <br>
Multiplicaremos por 100 para termos o valor em porcentagem e usaremos o comando **.round()** para arredondar o resultado


In [77]:
df = df.withColumn("alto_fech", col('maxima') ==col('fechamento'))
df.show()

+----+----------+--------+------+------+----------+--------+----------+---------+
|nome|      data|abertura|maxima|minima|fechamento|  volume|fech_abert|alto_fech|
+----+----------+--------+------+------+----------+--------+----------+---------+
| AAL|2013-02-08|   15.07| 15.12| 14.63|     14.75| 8407500|     false|    false|
| AAL|2013-02-11|   14.89| 15.01| 14.26|     14.46| 8882000|     false|    false|
| AAL|2013-02-12|   14.45| 14.51|  14.1|     14.27| 8126000|     false|    false|
| AAL|2013-02-13|    14.3| 14.94| 14.25|     14.66|10259500|      true|    false|
| AAL|2013-02-14|   14.94| 14.96| 13.16|     13.99|31879900|     false|    false|
| AAL|2013-02-15|   13.93| 14.61| 13.93|      14.5|15628000|      true|    false|
| AAL|2013-02-19|   14.33| 14.56| 14.08|     14.26|11354400|     false|    false|
| AAL|2013-02-20|   14.17| 14.26| 13.15|     13.33|14725200|     false|    false|
| AAL|2013-02-21|   13.62| 13.95|  12.9|     13.37|11922100|     false|    false|
| AAL|2013-02-22

In [78]:
df.select('alto_fech').where(df.alto_fech=="True").count()
porcentagem1 =  (7420/619040)*100
round(porcentagem1, 2)

1.2

Em qual dia a ação da Apple mais subiu até o fechamento, de forma absoluta?<br>
Para descobrirmos qual é o resultado desta pergunta, deveremos criar uma nova coluna **(maioralta)** que agrupará o resultado da coluna **maxima** subtraído da coluna **minima**. <br>
Após este passo, selecionameremos apenas as ações correspondenter a **AAPL**. <br>
Agora então iremos agrupar os dados por **data** e **nome**, utilizaremos o comando **max** para a coluna **maioralta** e orderaremos de forma decrescente.

In [79]:
df = df.withColumn("maioralta", col('maxima') -col('minima'))
df.show()

+----+----------+--------+------+------+----------+--------+----------+---------+----------+
|nome|      data|abertura|maxima|minima|fechamento|  volume|fech_abert|alto_fech| maioralta|
+----+----------+--------+------+------+----------+--------+----------+---------+----------+
| AAL|2013-02-08|   15.07| 15.12| 14.63|     14.75| 8407500|     false|    false|0.48999977|
| AAL|2013-02-11|   14.89| 15.01| 14.26|     14.46| 8882000|     false|    false|      0.75|
| AAL|2013-02-12|   14.45| 14.51|  14.1|     14.27| 8126000|     false|    false|0.40999985|
| AAL|2013-02-13|    14.3| 14.94| 14.25|     14.66|10259500|      true|    false| 0.6899996|
| AAL|2013-02-14|   14.94| 14.96| 13.16|     13.99|31879900|     false|    false| 1.8000002|
| AAL|2013-02-15|   13.93| 14.61| 13.93|      14.5|15628000|      true|    false|0.67999935|
| AAL|2013-02-19|   14.33| 14.56| 14.08|     14.26|11354400|     false|    false| 0.4800005|
| AAL|2013-02-20|   14.17| 14.26| 13.15|     13.33|14725200|     false

In [80]:
apple1 = df[df['nome']=='AAPL']

In [81]:
apple1.groupby("data", "nome") \
.agg(max("maioralta").alias("maioralta")) \
.sort(desc("maioralta")) \
.show()  

+----------+----+---------+
|      data|nome|maioralta|
+----------+----+---------+
|2015-08-24|AAPL|16.800003|
|2018-02-06|AAPL| 9.720001|
|2017-06-09|AAPL| 9.169998|
|2014-12-01|AAPL|7.9800034|
|2018-02-05|AAPL| 7.880005|
|2015-08-25|AAPL|7.6100006|
|2014-09-09|AAPL|6.9400024|
|2018-02-02|AAPL| 6.699997|
|2017-07-27|AAPL|6.6900024|
|2015-08-21|AAPL| 6.255005|
|2015-08-12|AAPL| 5.790001|
|2017-11-29|AAPL|5.7599945|
|2015-09-29|AAPL|5.6500015|
|2017-08-10|AAPL| 5.369995|
|2015-08-05|AAPL| 5.340004|
|2016-11-10|AAPL|5.2599945|
|2014-12-15|AAPL|     5.25|
|2017-09-12|AAPL|5.1900024|
|2015-08-03|AAPL| 5.050003|
|2015-04-28|AAPL| 4.969986|
+----------+----+---------+
only showing top 20 rows



Em média, qual o volume diário de transações das ações da AAPL? <br>
Podemos utilizar o comando **.describe()** ou podemos utilizar o **.groupBy**, **mean** e organizar por ordem descrescente.


In [82]:
apple1.describe().show()

+-------+----+------------------+------------------+------------------+------------------+-------------------+------------------+
|summary|nome|          abertura|            maxima|            minima|        fechamento|             volume|         maioralta|
+-------+----+------------------+------------------+------------------+------------------+-------------------+------------------+
|  count|1259|              1259|              1259|              1259|              1259|               1259|              1259|
|   mean|null|109.05542896625255|109.95111835107053|108.14158904277872|109.06669843944886|5.404789973550437E7|1.8095293082918222|
| stddev|null|30.549219948432892|30.686186467215954| 30.37622368001498| 30.55681154487995|  3.3468353335784E7| 1.117369805426913|
|    min|AAPL|           55.4242|           57.0857|           55.0142|           55.7899|           11475922|        0.45999908|
|    max|AAPL|            179.37|             180.1|            178.25|            179.26|

In [83]:
apple1.groupby("nome") \
.agg(mean("volume").alias("volume")) \
.sort(desc("volume")) \
.show() 

+----+-------------------+
|nome|             volume|
+----+-------------------+
|AAPL|5.404789973550437E7|
+----+-------------------+



Quantas ações tem 1, 2, 3, 4 e 5 caracteres em seu nome, respectivamente?


In [84]:
from pyspark.sql.functions import length

In [85]:
acoes = df.dropDuplicates(['nome'])


In [86]:
acoes1 = acoes.where(length(col("nome")) == 1).count()
acoes1


10

In [87]:
acoes2 = acoes.where(length(col("nome")) == 2).count()
acoes2


50

In [90]:
acoes3 = acoes.where(length(col("nome")) == 3).count()
acoes3


323

In [91]:
acoes4 = acoes.where(length(col("nome")) == 4).count()
acoes4

117

In [92]:
acoes5 = acoes.where(length(col("nome")) == 5).count()
acoes5

5

Qual a ação menos negociada da bolsa, em volume de transações?


In [93]:
df.groupBy("nome") \
  .agg(sum("volume").alias("volume")) \
  .sort(("volume")) \
  .show()

+----+---------+
|nome|   volume|
+----+---------+
|APTV| 92947779|
| BHF|144878392|
| MTD|214489754|
| AZO|397585127|
|WLTW|414734590|
| HII|418791148|
| DXC|424529395|
|  RE|434037215|
| ESS|477004832|
| SNA|530283843|
| FRT|531047903|
|ANSS|552950012|
| IFF|558773791|
| AMG|568888091|
| COO|569985137|
|  IT|578515652|
| ROP|579470940|
| AYI|608654295|
| FTV|610590073|
|BHGE|628313042|
+----+---------+
only showing top 20 rows



Com qual frequência o preço de fechamento é também o mais alto do dia?


In [95]:
df = df.withColumn("fech_alto", col('fechamento') ==col('maxima'))
df.show()

+----+----------+--------+------+------+----------+--------+----------+---------+----------+---------+
|nome|      data|abertura|maxima|minima|fechamento|  volume|fech_abert|alto_fech| maioralta|fech_alto|
+----+----------+--------+------+------+----------+--------+----------+---------+----------+---------+
| AAL|2013-02-08|   15.07| 15.12| 14.63|     14.75| 8407500|     false|    false|0.48999977|    false|
| AAL|2013-02-11|   14.89| 15.01| 14.26|     14.46| 8882000|     false|    false|      0.75|    false|
| AAL|2013-02-12|   14.45| 14.51|  14.1|     14.27| 8126000|     false|    false|0.40999985|    false|
| AAL|2013-02-13|    14.3| 14.94| 14.25|     14.66|10259500|      true|    false| 0.6899996|    false|
| AAL|2013-02-14|   14.94| 14.96| 13.16|     13.99|31879900|     false|    false| 1.8000002|    false|
| AAL|2013-02-15|   13.93| 14.61| 13.93|      14.5|15628000|      true|    false|0.67999935|    false|
| AAL|2013-02-19|   14.33| 14.56| 14.08|     14.26|11354400|     false|  

In [96]:
df.select('fech_alto').where(df.fech_alto=="True").count()

7420

In [97]:
porcentagem2 =  (7420/619040)*100
round(porcentagem2, 2)

1.2